### QB using Fantasy Pros stats

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
pd.options.display.max_columns = None # See all columns
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

The bulk of the preprocessing logic can be found in the source code.

##### To make our dataset, lets pull quarterbacks from the last ~10 years

In [12]:
QB_2021 = football.prepare_QB("2021")
QB_2020 = football.prepare_QB("2020")
QB_2019 = football.prepare_QB("2019")
QB_2018 = football.prepare_QB("2018")
QB_2017 = football.prepare_QB("2017")
QB_2016 = football.prepare_QB("2016")
QB_2015 = football.prepare_QB("2015")
QB_2014 = football.prepare_QB("2014")
QB_2013 = football.prepare_QB("2013")

In [13]:
all_QB = [QB_2021, QB_2020, QB_2019, QB_2018,
          QB_2017, QB_2016, QB_2015, QB_2014, QB_2013]

QB = pd.concat(all_QB, ignore_index=True)
QB

,CMP,Passing_att,PCT,Passing_Yds,Y/A,Passing_Td,INT,SACKS,Rushing_att,Rushing_Yds,Rushing_Td,FPTS/G,y,AIR/A,10+ YDS,20+ YDS,30+ YDS,40+ YDS,50+ YDS,PKT TIME,KNCK,HRRY,BLITZ,POOR,DROP,RZ ATT,RTG
0,24.058824,38.000000,63.3,259.235294,6.8,2.117647,0.882353,1.529412,7.176471,44.882353,0.352941,24.6,25.2,4.5,10.647059,3.000000,0.941176,0.470588,0.176471,2.4,3.705882,5.647059,10.705882,6.352941,2.058824,7.235294,5.411765
1,26.058824,39.529412,65.9,294.941176,7.5,2.235294,0.882353,1.823529,3.705882,17.764706,0.176471,23.3,24.3,4.3,12.235294,3.058824,1.294118,0.882353,0.176471,2.4,3.647059,2.941176,9.058824,5.588235,2.235294,6.117647,5.823529
2,28.529412,42.294118,67.5,312.705882,7.4,2.529412,0.705882,1.294118,1.647059,4.764706,0.117647,22.7,25.6,4.3,12.058824,4.411765,1.352941,0.588235,0.235294,2.3,1.764706,1.764706,10.411765,7.941176,1.882353,7.058824,6.000000
3,25.647059,38.705882,66.3,284.000000,7.3,2.176471,0.764706,1.647059,3.882353,22.411765,0.117647,22.0,21.7,3.8,11.294118,3.411765,1.235294,0.647059,0.235294,2.4,3.411765,4.470588,6.588235,6.823529,1.823529,5.882353,5.882353
4,23.764706,35.352941,67.2,287.411765,8.1,2.411765,1.000000,1.764706,1.882353,2.529412,0.000000,20.4,18.5,5.0,10.235294,3.823529,1.647059,1.058824,0.588235,2.4,2.470588,1.882353,8.117647,6.529412,1.823529,6.588235,6.176471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,18.000000,30.600000,58.8,197.200000,6.4,1.100000,0.900000,2.800000,5.300000,18.600000,0.200000,13.4,11.5,3.8,7.400000,2.800000,1.000000,0.400000,0.000000,0.0,0.000000,0.000000,8.400000,0.000000,1.900000,2.100000,8.000000
277,21.900000,35.800000,61.2,231.000000,6.5,1.000000,1.400000,2.100000,0.500000,2.400000,0.000000,10.5,14.9,4.2,9.900000,2.600000,0.900000,0.300000,0.000000,0.0,0.000000,0.000000,12.300000,0.000000,1.200000,4.600000,7.200000
278,14.200000,24.200000,58.7,167.300000,6.9,0.800000,0.700000,2.100000,2.300000,6.400000,0.000000,8.5,10.1,4.1,6.600000,2.200000,0.700000,0.200000,0.200000,0.0,0.000000,0.000000,7.300000,0.000000,1.400000,3.400000,8.200000
279,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,1.5,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
